In [80]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow import keras
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv


In [2]:
!nvidia-smi

Wed May 28 14:16:03 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   35C    P0             27W /  250W |       0MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [81]:

gpus = tf.config.list_physical_devices('GPU')

In [82]:
print(gpus)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [83]:
tf.device('/physical_device:GPU:0')

In [84]:
from sklearn.model_selection import train_test_split

In [154]:
data = pd.read_csv(r"/kaggle/input/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv")

In [8]:
def cat_to_bin(col_name):
    data[col_name] = data[col_name].where(data[col_name] == "Yes" , 1)
    data[col_name] = data[col_name].where(data[col_name] == "No" , 0)
    print("***Success***")

In [155]:
data["TotalCharges"] = pd.to_numeric(data["TotalCharges"] , errors = "coerce")

In [156]:
data.dtypes

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges        float64
Churn                object
dtype: object

In [157]:
data.drop(["customerID"] , axis = 1 , inplace = True)

In [158]:
tokenizer = keras.preprocessing.text.Tokenizer()

In [159]:
data.columns

Index(['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure',
       'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity',
       'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV',
       'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod',
       'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')

In [160]:
lst = [i for i in data.columns if data[i].dtype == "object"]

In [142]:
lst

['gender',
 'Partner',
 'Dependents',
 'PhoneService',
 'MultipleLines',
 'InternetService',
 'OnlineSecurity',
 'OnlineBackup',
 'DeviceProtection',
 'TechSupport',
 'StreamingTV',
 'StreamingMovies',
 'Contract',
 'PaperlessBilling',
 'PaymentMethod',
 'Churn']

In [161]:
data.dtypes

gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges        float64
Churn                object
dtype: object

In [162]:
for cols in data.columns:
    print(f"{cols} : {len(pd.unique(data[cols]))}")

gender : 2
SeniorCitizen : 2
Partner : 2
Dependents : 2
tenure : 73
PhoneService : 2
MultipleLines : 3
InternetService : 3
OnlineSecurity : 3
OnlineBackup : 3
DeviceProtection : 3
TechSupport : 3
StreamingTV : 3
StreamingMovies : 3
Contract : 3
PaperlessBilling : 2
PaymentMethod : 4
MonthlyCharges : 1585
TotalCharges : 6531
Churn : 2


In [163]:
encoder_list = []
for cols in lst:
    print(f"{cols} : {len(pd.unique(data[cols]))}")
    if len(pd.unique(data[cols])) > 2:
        lst.remove(cols)
        encoder_list.append(cols)

gender : 2
Partner : 2
Dependents : 2
PhoneService : 2
MultipleLines : 3
OnlineSecurity : 3
DeviceProtection : 3
StreamingTV : 3
Contract : 3
PaymentMethod : 4


In [165]:
print([data.columns])

[Index(['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure',
       'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity',
       'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV',
       'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod',
       'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')]


In [166]:
lst

['gender',
 'Partner',
 'Dependents',
 'PhoneService',
 'InternetService',
 'OnlineBackup',
 'TechSupport',
 'StreamingMovies',
 'PaperlessBilling',
 'Churn']

In [167]:
encoder_list

['MultipleLines',
 'OnlineSecurity',
 'DeviceProtection',
 'StreamingTV',
 'Contract',
 'PaymentMethod']

In [153]:
print(encoder_list)

['MultipleLines', 'OnlineSecurity', 'DeviceProtection', 'StreamingTV', 'Contract', 'PaymentMethod']


In [169]:
# data[[ 'gender', 'SeniorCitizen', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService',
#        'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
#        'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
#        'PaymentMethod' ]].astype(data[ 'gender', 'SeniorCitizen', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService',
#        'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
#        'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
#        'PaymentMethod' ]).cat.codes

def cat_conv(col_name):
    data[col_name] = data[col_name].astype('category').cat.codes
    print("Success")


for cols in lst:
    cat_conv(cols)


data = pd.get_dummies(data, columns=['MultipleLines', 'OnlineSecurity', 'DeviceProtection', 'StreamingTV', 'Contract', 'PaymentMethod'])

Success
Success
Success
Success
Success
Success
Success
Success
Success
Success


In [60]:
cm  = data.corr()

In [171]:
data = data.replace({"True" : 1 , "False" : 0})

In [174]:
data.isna().sum()

gender                                     0
SeniorCitizen                              0
Partner                                    0
Dependents                                 0
tenure                                     0
PhoneService                               0
InternetService                            0
OnlineBackup                               0
TechSupport                                0
StreamingMovies                            0
PaperlessBilling                           0
MonthlyCharges                             0
TotalCharges                               0
Churn                                      0
MultipleLines_No                           0
MultipleLines_No phone service             0
MultipleLines_Yes                          0
OnlineSecurity_No                          0
OnlineSecurity_No internet service         0
OnlineSecurity_Yes                         0
DeviceProtection_No                        0
DeviceProtection_No internet service       0
DeviceProt

In [173]:
data["TotalCharges"].fillna(data["TotalCharges"].mean() , inplace = True)

/tmp/ipykernel_35/2081533823.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data["TotalCharges"].fillna(data["TotalCharges"].mean() , inplace = True)


In [109]:
any(data.isna())

True

In [175]:
x = data.drop(["Churn"] , axis = 1)
y = data["Churn"]

In [176]:
x_train , x_test , y_train ,y_test = train_test_split(x,y , test_size = 0.33 , random_state = 45)

In [177]:
x_train.shape

(4718, 32)

In [190]:
model = keras.Sequential([keras.layers.Dense(64 , input_shape = (32,) , activation = "relu") , 
                          keras.layers.Dense(46 , activation = "tanh"),
                        keras.layers.Dense(1 , activation = "sigmoid")])
optimzer = keras.optimizers.Adam(learning_rate = 0.01)
model.compile(optimizer = optimzer , loss = "BinaryCrossentropy" , metrics = ["accuracy"])


In [191]:
model.fit(x_train , y_train , epochs = 50 , validation_data = (x_test , y_test))

Epoch 1/50
148/148 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.7254 - loss: 0.5666 - val_accuracy: 0.7708 - val_loss: 0.5330
Epoch 2/50
148/148 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7489 - loss: 0.5490 - val_accuracy: 0.7583 - val_loss: 0.5428
Epoch 3/50
148/148 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7526 - loss: 0.5372 - val_accuracy: 0.7776 - val_loss: 0.5436
Epoch 4/50
148/148 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7531 - loss: 0.5631 - val_accuracy: 0.7768 - val_loss: 0.5242
Epoch 5/50
148/148 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7255 - loss: 0.5932 - val_accuracy: 0.7454 - val_loss: 0.5678
Epoch 6/50
148/148 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7268 - loss: 0.5824 - val_accuracy: 0.7454 - val_loss: 0.5690
Epoch 7/50
148/148 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7294 - loss: 0.5904 - val_accuracy: 0.7454 - val_loss: 0.5826
Epoch 8/50
148/148 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7171 - loss: 0.5993 - val_accuracy: 0.

In [183]:
model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_20 (Dense)                     │ (None, 640)                 │          21,120 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_21 (Dense)                     │ (None, 46)                  │          29,486 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_22 (Dense)                     │ (None, 1)                   │              47 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 151,961 (593.60 KB)

 Trainable params: 50,653 (197.86 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 101,308 (395.74 KB)